# Training the model to run on Lichess data
## Some pre-requisites if running on Google Collab
If not running on Google collab do not run these next two cells!

In [ ]:
# Install the only dependency not available from collab directly
!pip install chess

# Get imported files from repo
!git clone -b rl-setup https://github.com/owenjaques/chessbot.git
!mv chessbot chessbot-repo
!mv chessbot-repo/src/chessbot .
!rm chessbot-repo -r

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
weights_directory = '/content/gdrive/MyDrive/chessbot_weights/'
print(f'Saving weights to {weights_directory}')

## If not running on Google Collab
Set the weights directory variable to wherever you would like data saved.

In [ ]:
weights_directory = 'your directory here'

## Get the data
This compression format is really nice, so you can cancel this cell whenever you want and all the games that were downloaded will be maintained. In my experience 300Mb gets well over 100,000 games.

In [ ]:
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2023-02.pgn.zst

## Decompress the Data

In [ ]:
!apt install zstd
!pzstd -d lichess_db_standard_rated_2023-02.pgn.zst

## Play the games from the data
For this section we want to play the games so we can translate them into labelled model inputs which we can train on. 

In [ ]:
import chess
import chess.pgn
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
from collections import deque
from chessbot.model_input import ModelInput

X_all = deque()
y_all = deque()

MAX_GAMES = 100000

with open('lichess_db_standard_rated_2023-02.pgn') as pgn:
    game_count = 0
    game = chess.pgn.read_game(pgn)

    while game is not None and game_count < MAX_GAMES:
        result = game.headers['Result']
        
        # Only train on game played to completion that were not draws
        if game.headers['Termination'] == 'Normal' and result in ['1-0', '0-1']:
            print(f'\rProcessing game {game_count}/{MAX_GAMES}', end='')
            X = []
            
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                X.append(ModelInput(board).get_input())

            X = np.array(X)
            y = np.empty_like(X)
            
            discount_factor = 0.95
            y_reversed_indices = np.linspace(len(y) - 1, 0, num=len(y))
            y = 1 * discount_factor**y_reversed_indices

            if result == '0-1':
                y = -y

            # Scale the labels to be between 0 and 1 instead of -1 and 1
            y = (y + 1) / 2

            # Save the data
            if X is not None and y is not None:
                try:
                    X_all.extend(X)
                    y_all.extend(y)
                except:
                    pass

        # Get the next game
        game_count += 1
        game = chess.pgn.read_game(pgn)

        # Save current game data every 10000 games
        if game_count % 10000 == 0:
            X = np.array(X_all)
            y = np.array(y_all)
            np.savez_compressed(f'{weights_directory}games_data.npz', X=X, y=y)

### Load data from previous step
If your kernel has restarted for some reason, running this cell will load the games from the last step that were saved to disk.

In [ ]:
data = np.load(f'{weights_directory}games_data.npz')
X = data['X']
y = data['y']

### Experiment to balance training set
If you look at the first histogram in this experiment you will see that the training set is unbalanced, this cell aims to balance the training set by undersampling it.

In [ ]:
n_bins = 50

fig, ax = plt.subplots()
ax.hist(y, bins=n_bins)
ax.set_title('Distribution of training labels')
ax.set_xlabel('label (y)')
ax.set_ylabel('no. of occurences in dataset')
plt.show()

bins = np.linspace(0, 1, n_bins + 1)
binned_y_indices = np.digitize(y, bins)
unique_bins, counts = np.unique(binned_y_indices, return_counts=True)
n_to_sample = np.min(counts)

mask = np.full(y.shape[0], False, dtype=bool)
for i in range(y.shape[0]):
    if n_to_sample / counts[binned_y_indices[i] - 2] > np.random.rand():
        mask[i] = True

X_resampled = X[mask]
y_resampled = y[mask]

fig, ax = plt.subplots()
ax.hist(y_resampled, bins=n_bins)
ax.set_title('Distribution of training labels (undersampled)')
ax.set_xlabel('label (y)')
ax.set_ylabel('no. of occurences in dataset')
plt.show()

X = X_resampled
y = y_resampled

## Our model
Set up your model being used here.

In [ ]:
model = keras.Sequential([
	keras.layers.Dense(1024, activation='relu'),
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
	metrics=[keras.metrics.MeanAbsoluteError()]
)

## Training the model
This next cell trains the model on the training data, then saves it to disk. Note multiple calls to this cell have crashed the notebook before due to high RAM usages.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', restore_best_weights=True, patience=3, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=0.00001, verbose=1)
model.fit(X, y, epochs=100, batch_size=1024, callbacks=[early_stopping, reduce_lr])
model.save(f'{weights_directory}lichess_trained_model')

## Why not play a game after all that training?

In [ ]:
import time
from IPython.display import clear_output
from chessbot.chessbot import ChessBot

def play_game(model, exploration_rate=0.0, should_visualise=False):
	white = ChessBot(model, chess.WHITE, exploration_rate)
	black = ChessBot(model, chess.BLACK, exploration_rate)

	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over(claim_draw=True):
		board.push(black.move(board) if board.turn == chess.BLACK else white.move(board))

		if should_visualise:
			clear_output(wait=True)
			display(board)
			time.sleep(0.5)

	return board.outcome(claim_draw=True).result()
 
play_game(model, should_visualise=True)